In [ ]:
%reset -f

In [ ]:
#ativar a venv -> .\.venv\Scripts\Activate.ps1
#desativar a venv -> deactivate
import pandas as pd 
import numpy as np 
from pathlib import Path

bases_path = Path("bases_B")
output =  Path("OUTPUT")
output.mkdir(exist_ok=True)

arquivo_listagem = bases_path / "listagem.xlsx"
arquivo_checklist = bases_path / "CHECKLIST_UNIFICADO_20251128_104257.xlsx"
fases = bases_path / "fase.xlsx"

df_listagem = pd.read_excel(arquivo_listagem, sheet_name='Lojas FRQ', engine="calamine")
df_checklist = pd.read_excel(arquivo_checklist, engine="calamine")
df_fases = pd.read_excel(fases, engine="calamine")


In [ ]:
df_checklist.head()

In [ ]:
df_checklist.columns

In [ ]:
df_listagem.head()

In [ ]:
df_listagem.columns

In [ ]:
df_fases.head()

In [ ]:
def validar_checklist(df_listagem, df_checklist, df_fases):
    
   #criar uma coluna uxiliar de alvo calculado = velocidade de venda* (leadtime + cobertura) com duas cada decimais
    df_checklist['Alvo Calculado'] = df_checklist['VELOCIDADE_VENDA'] * (df_checklist['LEADTIME'] + df_checklist['COBERTURA'])
    df_checklist['Alvo Calculado'] = df_checklist['Alvo Calculado'].round(2)
    # alvo final = pegar o máximo entre a coluna alvo calculado e a coluna alvo minimo e arredondar para cima o valor máximo entre os dois
    df_checklist['Alvo Final'] = np.maximum(df_checklist['Alvo Calculado'], df_checklist['MINIMO']).apply(np.ceil) 
   # validar se a coluna alvo final é igual a coluna ALVO, se sim retornar sim senão não
    df_checklist["Validação Alvo"] = np.where(df_checklist["Alvo Final"] == df_checklist["ALVO"], 'SIM', 'NÃO')
     # concatenar a coluna PRODUTO  e a coluna COR
    df_checklist['Artigo Cor'] = df_checklist['PRODUTO'].astype(str)  + df_checklist['COR'].astype(str)
    # validar a coluna MINIMO se é maior que coluna alvo calculado, se sim retornar sim senão não
    df_checklist['Validação MINIMO'] = np.where(df_checklist['MINIMO'] > df_checklist['Alvo Calculado'], 'SIM', 'NÃO')
    #validar se a coluna NECESSIDADE é maior ou igual a coluna de RESSUPRIR
    df_checklist['Validação RESSUPRIR'] = np.where(df_checklist['NECESSIDADE'] >= df_checklist['RESSUPRIR'], 'SIM', 'NÃO')
    # fazer um merge entre o df_checklist e o df_listagem pegar as colunas Franqueado, Pot.Venda, Sortimento
    df_checklist = df_checklist.merge(df_listagem[['FILIAL', 'Franqueado', 'Pot. Venda', 'Sortimento Ajustado']], on='FILIAL', how='left')
    #Se o sortimento não for encontrado retorne 0
    df_checklist['Sortimento Ajustado'] = df_checklist['Sortimento Ajustado'].fillna(0)
    #validar a coluna NECESSIDADE se é maior que 0
    df_checklist['NECESSIDADE ?'] = np.where(df_checklist['NECESSIDADE'] > 0, 'SIM', 'NÃO')
    #validar se a coluna NECESSIDADE é igual a se a coluna ALVO - RESERVADO - EMBALOADO for menor que 0 é 0, senão ALVO - RESERVADO - EMBALOADO E COMPARAR COM A COLUNA NECESSIDADE e se for igual retorne sim senão não
    necessidade_calculada = df_checklist['ALVO'] - df_checklist['PRESENTE'] - df_checklist['EMBALADO'] - df_checklist['RESERVADO']
    necessidade_calculada = necessidade_calculada = np.maximum(0, necessidade_calculada)
    df_checklist['Validação NECESSIDADE'] = np.where(df_checklist['NECESSIDADE'] == necessidade_calculada, 'SIM', 'NÃO')
    # concatenar a coluna FILIAL + - + ARTIGO_COR
    df_checklist['Filial-Artigo_Cor'] = df_checklist['FILIAL'].astype(str) + ' - ' + df_checklist['ARTIGO_COR'].astype(str)
    # fazer um merge entre o df_checklist e o df_fases e pegar a coluna fase
    df_checklist = df_checklist.merge(df_fases[['Artigo Cor', ' fase']], on='Artigo Cor', how='left')

    return  df_checklist # Retornar True se todas as validações passarem

In [ ]:
#exibir e chamar a função validar_checklist
df_resultado = validar_checklist(df_listagem, df_checklist.copy(), df_fases)
df_resultado.head()


In [ ]:
df_resultado.columns

In [ ]:
# ... existing code ...
colunas_selecionadas = [
    'Alvo Calculado', 'Alvo Final', 'Artigo Cor', 'Validação MINIMO', 
    'Validação RESSUPRIR', 'Artigo Cor', 'Franqueado', 'Pot. Venda', 
    'Sortimento Ajustado', 'NECESSIDADE ?', 'Validação NECESSIDADE', 
    'Filial-Artigo_Cor', ' fase', 'Validação Alvo'
]

# Select the columns using a list
df_selecionado = df_resultado[colunas_selecionadas]

df_selecionado.isnull().sum()

In [ ]:
filiais_sem_franqueado = df_resultado[df_resultado['Franqueado'].isnull()]['FILIAL']
print(filiais_sem_franqueado.unique())

In [ ]:
#exibir a coluna DATA da validação checklist
df_resultado['DATA'].head(1)


In [ ]:
# Define o nome do arquivo e da aba de saída
nome_arquivo_saida = "(Validação) 35ª rodada_24.11.xlsx"
caminho_saida = output / nome_arquivo_saida
nome_aba = "Base rep."

print(f"Salvando resultado em '{nome_arquivo_saida}'...")

 #Salva o DataFrame no arquivo Excel especificado
df_resultado.to_excel(caminho_saida, sheet_name=nome_aba, index=False)

print(f"Processo concluído! Arquivo salvo em: {caminho_saida}")